In [0]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
import pandas as pd
import numpy as np

'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''Display'''
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

In [0]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import itertools, string, operator, re, unicodedata, nltk
from operator import itemgetter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from bs4 import BeautifulSoup
import numpy as np
from itertools import combinations
from gensim.models import Phrases
from collections import Counter
from sklearn.externals import joblib 
import pickle

Functions for Data-cleaning and Pre-processing

In [0]:
#Contraction map
c_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have",
  "no_text": " "
}

c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

add_stop = ['', ' ', 'say', 's', 'u', 'ap', 'afp', '...', 'n', '\\']

stop_words = ENGLISH_STOP_WORDS.union(add_stop)

tokenizer = TweetTokenizer()
pattern = r"(?u)\b\w\w+\b" 

lemmatizer = WordNetLemmatizer()

punc = list(set(string.punctuation))

def casual_tokenizer(text): #Splits words on white spaces (leaves contractions intact) and splits out trailing punctuation
    tokens = tokenizer.tokenize(text)
    return tokens

#Function to replace the nltk pos tags with the corresponding wordnet pos tag to use the wordnet lemmatizer
def get_word_net_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemma_wordnet(tagged_text):
    final = []
    for word, tag in tagged_text:
        wordnet_tag = get_word_net_pos(tag)
        if wordnet_tag is None:
            final.append(lemmatizer.lemmatize(word))
        else:
            final.append(lemmatizer.lemmatize(word, pos=wordnet_tag))
    return final

def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

def remove_html(text):
    soup = BeautifulSoup(text, "html5lib")
    tags_del = soup.get_text()
    uni = unicodedata.normalize("NFKD", tags_del)
    bracket_del = re.sub(r'\[.*?\]', '  ', uni)
    apostrphe = re.sub('’', "'", bracket_del)
    string = apostrphe.replace('\r','  ')
    string = string.replace('\n','  ')
    extra_space = re.sub(' +',' ', string)
    return extra_space

def process_text(text):
    soup = BeautifulSoup(text, "lxml")
    tags_del = soup.get_text()
    no_html = re.sub('<[^>]*>', '', tags_del)
    tokenized = casual_tokenizer(no_html)
    lower = [item.lower() for item in tokenized]
    decontract = [expandContractions(item, c_re=c_re) for item in lower]
    tagged = nltk.pos_tag(decontract)
    lemma = lemma_wordnet(tagged)
    no_num = [re.sub('[0-9]+', '', each) for each in lemma]
    no_punc = [w for w in no_num if w not in punc]
    no_stop = [w for w in no_punc if w not in stop_words]
    return no_stop

def word_count(text):
    return len(str(text).split(' '))

def word_freq(clean_text_list, top_n):
    """
    Word Frequency
    """
    flat = [item for sublist in clean_text_list for item in sublist]
    with_counts = Counter(flat)
    top = with_counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T

def word_freq_bigrams(clean_text_list, top_n):
    """
    Word Frequency With Bigrams
    """
    bigram_model = Phrases(clean_text_list, min_count=2, threshold=1)
    w_bigrams = list(bigram_model[clean_text_list])
    flat_w_bigrams = [item for sublist in w_bigrams for item in sublist]
    with_counts = Counter(flat_w_bigrams)
    top = with_counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T


def bigram_freq(clean_text_list, top_n):
    bigram_model = Phrases(clean_text_list, min_count=2, threshold=1)
    w_bigrams = list(bigram_model[clean_text_list])
    flat_w_bigrams = [item for sublist in w_bigrams for item in sublist]
    bigrams = []
    for each in flat_w_bigrams:
        if '_' in each:
            bigrams.append(each)
    counts = Counter(bigrams)
    top = counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T

In [0]:
df = pd.read_csv('Posts-data.csv')

Pre-process 'text' field of our data. We need to eliminate 'no_text' while cleaning. So we replaced 'no_text' with space in c_dict.

In [0]:
df['clean_text'] = df['text'].apply(process_text)
df.head()

,text,flair,clean_text
0,Ordering and buying stuff online right now Hi!...,AskIndia,"[ordering, buying, stuff, online, right, hi, a..."
1,Are there any astrologers on reddit? Did you w...,AskIndia,"[astrologer, reddit, warn, regular, customer, ..."
2,Is there any good smartphone with 3 or even 4 ...,AskIndia,"[good, smartphone, sim, card, hello, look, goo..."
3,Required: Medium of Instruction Letter (MOI) f...,AskIndia,"[require, medium, instruction, letter, moi, mu..."
4,American Wanting to Travel to India - Tips and...,AskIndia,"[american, want, travel, india, tip, advice, h..."


In [0]:
# Counting word-frequency or most occuring words in clean-text field.
cl_text_list = df['clean_text'].tolist() 
wf = word_freq(cl_text_list, 20)
wf.head(20)

,0,1
0,no_text,652
1,india,352
2,’,232
3,indian,221
4,time,155
5,people,151
6,covid,125
7,help,120
8,like,113
9,make,113


In [0]:
df['word_count'] = df['text'].apply(word_count)
avg_wc = df.groupby('flair').mean().reset_index()
avg_wc[['flair','word_count']]

,flair,word_count
0,AskIndia,120.05
1,Business/Finance,49.08
2,Coronavirus,22.01
3,Food,33.34
4,Non-Political,25.38
5,Photography,13.58
6,Policy/Economy,103.26
7,Politics,40.22
8,Scheduled,110.67
9,Science/Technology,65.27


In [0]:
LE = LabelEncoder()
df['flair_num'] = LE.fit_transform(df['flair'].astype(str))

In [0]:
df.flair_num.value_counts()

6     122
7     120
4     120
2     120
0     120
5      67
1      59
9      37
11     34
10     34
3      29
8      21
Name: flair_num, dtype: int64

In [0]:
df.flair.value_counts()

Policy/Economy        122
Non-Political         120
Politics              120
Coronavirus           120
AskIndia              120
Photography            67
Business/Finance       59
Science/Technology     37
[R]eddiquette          34
Sports                 34
Food                   29
Scheduled              21
Name: flair, dtype: int64

Tfidf vectorizer is used for building features from our clean_text field. For building features we are using unigrams and bigrams only. ngram_range = (1, 2)

In [0]:
texts = df['clean_text'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X = tfidf_vectorizer.fit_transform(texts) #features
y = df['flair_num'].values #target

print (X.shape)
print(y.shape)

(883, 9322)
(883,)


As our clean_text data field is not very big (consists majorly title data), we are truncating features to 10 features only. So, that later while deploying our model it can work properly.

In [0]:
lsa = TruncatedSVD(n_components=10, 
                   n_iter=10, 
                   random_state=3)

X = lsa.fit_transform(X)
X.shape

(883, 10)

Testing which Scikit ML model works best on our dataset. We built a dictionry of seven ml models.

In [0]:
model_dict = {'Dummy' : DummyClassifier(random_state=3),
              'Stochastic Gradient Descent' : SGDClassifier(random_state=3, loss='log'),
              'Random Forest': RandomForestClassifier(random_state=3), 
              'Decsision Tree': DecisionTreeClassifier(random_state=3),
              'AdaBoost': AdaBoostClassifier(random_state=3), 
              'Gaussian Naive Bayes': GaussianNB(),
              'K Nearest Neighbor': KNeighborsClassifier()}

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, shuffle = True, stratify = y, random_state = 3)
# Expliciting data into trainting and test data.

Compile all the model and result accuracy for each of the model.

In [0]:
def model_score_df(model_dict):
  model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], [] 
  for k,v in model_dict.items():
    model_name.append(k) 
    v.fit(X_train, y_train) 
    y_pred = v.predict(X_test) 
    ac_score_list.append(accuracy_score(y_test, y_pred)) 
    p_score_list.append(precision_score(y_test, y_pred, average='macro')) 
    r_score_list.append(recall_score(y_test, y_pred, average='macro')) 
    f1_score_list.append(f1_score(y_test, y_pred, average='macro')) 
    model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T 
    model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score'] 
    model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False) 
  return model_comparison_df

model_score_df(model_dict)

,model_name,accuracy_score,precision_score,recall_score,f1_score
2,Random Forest,0.58,0.66,0.53,0.55
3,Decsision Tree,0.51,0.50,0.48,0.48
6,K Nearest Neighbor,0.41,0.40,0.38,0.37
5,Gaussian Naive Bayes,0.37,0.38,0.33,0.32
1,Stochastic Gradient Descent,0.43,0.36,0.35,0.32
4,AdaBoost,0.26,0.15,0.24,0.17
0,Dummy,0.08,0.05,0.06,0.05


As we can see from the above table, Random_Forest ML model is working best.

Train our Random_Forest model and exporting model as pickle file.

In [0]:
RFC = RandomForestClassifier(random_state=3)
RFC.fit(X_train, y_train)
pickle_out = open("model.pickle","wb")
pickle.dump(RFC, pickle_out)
pickle_out.close()
pickle_in = open("model.pickle","rb")
RFC_from_pickle = pickle.load(pickle_in)
RFC_from_pickle.predict(X_test)

array([ 2,  7,  7,  0,  6,  7,  2, 10,  0,  4,  5,  4, 11,  6,  0,  4,  6,
        7,  2,  2,  0,  7,  7,  4,  4,  2,  5,  5,  7,  4,  6,  8,  6,  8,
        8,  0,  2,  7,  1,  8,  4,  2,  9,  7,  1,  6,  6,  1,  6,  4,  1,
        0,  1,  0,  5,  2,  4,  0,  6,  4,  0,  7,  5,  7,  4,  0,  4, 11,
        0,  4,  4,  0,  2,  0,  6,  6,  9,  6,  0,  0,  4,  2,  4,  7,  4,
        6,  0,  0,  5,  2,  7,  4,  0,  0,  7,  0,  2,  6,  7,  1,  0,  5,
        4,  8,  6,  0, 10, 11,  4,  5,  4,  7,  0,  0,  6,  5,  2,  6,  2,
        0,  6,  4,  7,  5,  0,  4,  3,  2,  4,  6,  0,  5,  2, 10,  5,  7,
        4,  4,  6,  6,  7,  0,  0,  7,  0,  4,  4,  7,  0,  7,  8,  2,  6,
        2,  7,  1,  2,  4,  6,  7,  6,  7,  6,  0,  5,  6,  4,  4,  2,  2,
        2,  2,  4,  0,  6,  2,  2,  4, 11,  7,  0,  2,  4,  4,  9,  7,  7,
       11,  6,  4,  1,  1,  0,  4,  2,  2,  6,  7,  2,  6,  7, 11,  5,  6,
        4,  6,  4,  7,  7,  2,  5,  9,  0,  7,  2,  1,  7,  4,  7,  6,  2,
        4,  6,  0,  7,  4